In [1]:
options(warn=-1)

In [ ]:
# instala o pacote sqldf # insta 
# INSTALLING FROM CRAN
# Executar apenas na primeira vez
install.packages("sqldf")

In [ ]:
install.packagesinstall ("data.table")
install.packages("DataExplorer")

library(data.table)
library(DataExplorer)

In [ ]:
source("https://raw.githubusercontent.com/claudioteix/RLibrary/master/MyLibrary.R")

In [ ]:
library(sqldf)

In [ ]:
# Carrega o dataset criado no RStudio no servidor CEFET
# já com todas as cláusulas de data cleaning 
load("flights.Rda")

In [ ]:
nrow(df10)

In [ ]:
head(df10)

In [ ]:
str(df10)

In [ ]:
# Transforma o formato POSIXlt em chr
# para que possa ser trabalhado em conjunto com o sqldf
df10$depart_expect <- as.character(df10$depart_expect)
df10$depart <- as.character(df10$depart)
df10$arrival_expect <- as.character(df10$arrival_expect)
df10$arrival <- as.character(df10$arrival)

In [ ]:
str(df10)

In [ ]:
query<-"select destiny, origin, airline, flight, depart, arrival, departure_delay, arrival_delay,"
query<-paste(query,"depart_temperature, depart_dew_point, depart_humidity, depart_pressure, depart_visibility, depart_conditions,", sep=" ")
query<-paste(query,"arrival_temperature, arrival_dew_point, arrival_humidity, arrival_pressure, arrival_visibility, arrival_conditions", sep=" ")
query<-paste(query, "from df10", sep=" ")
print(query)

In [ ]:
train_set<-sqldf(query)

In [ ]:
str(train_set)

In [ ]:

# Carrega as bibliotecas necessárias para trabalhar com as transações e os algoritmos# Carre 
library("arules")
library("arulesSequences")
library("arulesViz")

In [ ]:
# Remove as características que não serão analisadas
train_set$depart_temperature <- NULL
train_set$depart_dew_point <- NULL
train_set$depart_humidity <- NULL
train_set$depart_pressure <- NULL
train_set$depart_visibility <- NULL
train_set$arrival_temperature <- NULL
train_set$arrival_dew_point <- NULL
train_set$arrival_humidity <- NULL
train_set$arrival_pressure <- NULL
train_set$arrival_visibility <- NULL

In [ ]:
str(train_set)

In [ ]:

# cria duas listas com todos os valores da coluna depart (index column=5)# cria  
# já separados nos formatos YYYY-MM-DD e HH:MM:SS respectivamente
depart_d<-sapply(strsplit(train_set[,5], " ", fixed = TRUE), "[", 1)
depart_h<-sapply(strsplit(train_set[,5], " ", fixed = TRUE), "[", 2)

# cria duas listas com todos os valores da coluna arrival (index column=6)
# já separados nos formatos YYYY-MM-DD e HH:MM:SS respectivamente
arrival_d<-sapply(strsplit(train_set[,6], " ", fixed = TRUE), "[", 1)
arrival_h<-sapply(strsplit(train_set[,6], " ", fixed = TRUE), "[", 2)

# cria um novo dataframe com essas duas novas colunas
x <- data.frame("depart_date" = depart_d, "depart_time" = depart_h, "arrival_date" = arrival_d, "arrival_time" = arrival_h)

# cria um novo dataframe com a combinação do dataframe train_set e x
flights <- cbind(train_set,x)

In [ ]:
str(flights)


In [ ]:
head(flights)

In [ ]:
# Remove as características que não serão analisadas
flights$depart <- NULL
flights$arrival <- NULL

In [ ]:
str(flights)

In [ ]:
introduce(flights)

In [ ]:
PlotMissing(flights)

In [ ]:
BarDiscrete(flights$origin)

In [ ]:
counts <- table(flights$depart_time)
barplot(counts, main="Flights Distribution by Departure Time", 
  	xlab="Departure Time")

In [ ]:
HistogramContinuous(flights)

In [ ]:
CorrelationDiscrete(flights)

In [ ]:
CorrelationContinuous(flights)

In [ ]:
counts <- table(flights$departure_delay)
barplot(counts, main="Flights Distribution by Departure Delay", 
  	xlab="Departure Delay")

In [ ]:
# Sum the flights delayed by departure time (HH)
query<-"select depart_time_HH, count(flight) as total_flights
        from (select substr(depart_time,1,2) as depart_time_HH, flight
              from   flights) 
        group by depart_time_HH
        order by total_flights"
print(query)

In [ ]:
result<-sqldf(query)

In [ ]:
result

In [ ]:
# Analisa a estatística descritiva das características departure_delay e arrival_delay que serão categorizadas
summary(flights$departure_delay)
summary(flights$arrival_delay)

In [ ]:
flights$depart_conditions <- as.factor(flights$depart_conditions)
flights$arrival_conditions <- as.factor(flights$arrival_conditions)

In [ ]:
str(flights)


**Departure Time** is a key factor that we are going to examine.
We want to know which time is the best time for flight.

In [ ]:
## Categoriza as variáveis departure_delay e arrival_delay
flights$departure_delay <- ordered(cut(flights$departure_delay, c(60,75,125,240)),
                              labels = c("None", "Low", "High"))

flights$arrival_delay <- ordered(cut(flights$arrival_delay, c(60,75,125,240)),
                              labels = c("None", "Low", "High"))

In [ ]:
## Categoriza a variável depart_time
flights$depart_time <- ordered(cut(as.integer(substr(flights$depart_time,1,2)), c(00,06,12,18,23)),
                              labels = c("Overnight", "Morning", "Afternoon", "Evening"))

In [ ]:
head(flights)


In [ ]:

# Sum the flights delayed during Overnight# Sum the 
query<-"select count(flight) from flights where depart_time='Overnight'"
print(query)

In [ ]:
result<-sqldf(query)


In [ ]:
result

In [ ]:
# Sum the flights delayed during Overnight and departure_delay='None'
query<-"select count(flight) from flights where depart_time='Overnight' and departure_delay='None'"
print(query)

In [ ]:
result_2<-sqldf(query)


In [ ]:
result_2

In [ ]:
# Sum the flights delayed during Morning
query<-"select count(flight) from flights where depart_time='Morning'"
result_3<-sqldf(query)
result_3

In [ ]:
# Sum the flights delayed during Morning and departure_delay='None'
query<-"select count(flight) from flights where depart_time='Morning' and departure_delay='None'"
result_4<-sqldf(query)
result_4

In [ ]:

# Sum the flights delayed during Afternoon# Sum the 
query<-"select count(flight) from flights where depart_time='Afternoon'"
result_5<-sqldf(query)
result_5

In [ ]:
# Sum the flights delayed during Afternoon and departure_delay='None'
query<-"select count(flight) from flights where depart_time='Afternoon' and departure_delay='None'"
result_6<-sqldf(query)
result_6

In [ ]:

# Sum the flights delayed during Evening# Sum the 
query<-"select count(flight) from flights where depart_time='Evening'"
result_7<-sqldf(query)
result_7

In [ ]:
# Sum the flights delayed during Evening and departure_delay='None'
query<-"select count(flight) from flights where depart_time='Evening' and departure_delay='None'"
result_8<-sqldf(query)
result_8

In [ ]:
## Cria as transações
train_set_transactions <- as(flights, "transactions")

In [ ]:
train_set_transactions

In [ ]:
str(train_set_transactions)


In [ ]:
head(train_set_transactions@data)
head(train_set_transactions@itemInfo)
head(train_set_transactions@itemsetInf

In [ ]:
# Nesta parte do trabalho, executar-se-á o algoritmo APRIORI
# para gerar regras com a seguinte especificcação:
#
# 1. com suporte de 0,5, confiança de 0,9 e tamanho minimo = 2
# 2. com um atributo do lado esquerdo e um do lado direito e tamanho maximo=3
# 3. com lado direito limitado ao valor do atributo arrival_delay=None Arrival Delay
# 4. e lado esquerdo livre.
#
# Isso significa que estamos interessados em qualquer causa que provoque
# a consequência arrival_delay=High Arrival Delay. 
#rules <- apriori(train_set_transactions, parameter=list(supp = 0.01, conf = 0.8, minlen=1, maxlen= 10, target = "rules"), appearance=list(rhs = c("arrival_delay=High"),default="lhs"),control=NULL)
#rules <- apriori(train_set_transactions, parameter=list(conf = 0.9, minlen=1, maxlen= 10, target = "rules"), appearance=list(rhs = c("arrival_delay=None"),default="lhs"),control=NULL)
#rules <- apriori(train_set_transactions, parameter = list(supp = 0.9, conf = 0.9, target = "rules"), appearance=list(rhs = c("arrival_delay=High"),default="lhs"),control=NULL)
rules <- apriori(train_set_transactions, parameter = list(supp = 0.01, conf = 0.1, target = "rules"), appearance=list(lhs = c("depart_time=Evening"), rhs = c("departure_delay=None")),control=NULL)
rules_a <- as(rules, "data.frame")
head(rules_a)

In [ ]:
irules <- inspect(rules)


In [ ]:
rules.sorted <- sort(rules, by="lift")
inspect(rules.sorted)

Interpretando o resultado da primeira regra, temos:

SUPPORT = 0,015 significa que 1,5% das 53.705 transações, representam o padrão {airline=WEB, departure_delay=High}

CONFIDENCE = 0,92 significa que 92% das 53.705 transações que contém o padrão {airline=WEB, departure_delay=High} também contém o padrão {arrival_delay=High}

O lift de uma regra de associação A Þ B indica o quanto mais freqüente torna-se B, quando A ocorre.

Esta medida é computada por: Lift(A Þ B) = Conf(A Þ B) ÷ Sup(B).

Indica que quando ocorre um voo com o padrão {airline=WEB, departure_delay=High} existe chance 4(quatro) vezes maior de ocorrer o padrão {arrival_delay=High}

{arrival_delay=High} significa um atraso entre 2 até 4 horas.

##############################################################################################################
##############################################################################################################


In [3]:
load("dataset/vrawu17.RData")

# Analisando os atrasos de partida

Para esta análise utilizaremos atrasos, de voos realizados, em um intervalo de 15m até 4h, pois são os atrasos mais comuns, os maiores que 4 horas apresentão consequências mais severas as empresas de aviação, de acordo com a ANAC as empresas tem que tomar as seguintes providências:

> - **A partir de 1 hora**: comunicação (internet, telefonemas, etc).
>- **A partir de 2 horas**: alimentação (voucher, lanche, bebidas, etc).
>- **A partir de 4 horas**: acomodação ou hospedagem (se for o caso) e transporte do aeroporto ao local de acomodação. Se você estiver no local de seu domicílio, a empresa poderá oferecer apenas o transporte para sua residência e desta para o aeroporto.


> **O passageiro pode solicitar reembolso quando ocorrer:**
> - a) Atraso do voo por mais de quatro horas;



Fonte: http://www.anac.gov.br/assuntos/passageiros/atrasos-cancelamento-e-pretericao-de-embarque

In [17]:
vw_c = vrawu17[,c("icao_origem","atraso_partida","situacao_voo")]
vw_c = vw_c[rowSums(is.na(vw_c))==0,]

In [19]:
dim(vw_c)

[1] 929924      3

In [20]:
atrasos = vw_c[vw_c$atraso_partida >15 & 
                  vw_c$atraso_partida <=240 & 
                  vw_c$situacao_voo=="REALIZADO",]

library(dplyr) #%>%

#count atrasos realziados
vw_g = atrasos[c("icao_origem")]
vw_g = vw_g %>% group_by(icao_origem) %>% summarize(Count = n())

#count totais realizados
vw_t = vw_c[vw_c$situacao_voo=="REALIZADO",c("icao_origem")]
vw_t = vw_t %>% group_by(vw_t) %>% summarize(Count = n())

vw_m = merge(vw_g,vw_t, by ='icao_origem')
head(vw_m)

In [ ]:
x = atrasos$partida_real
y = atrasos$atraso_partida
plot(x,y,pch=16 , cex=0.3, ylab='Atraso (min)', xlab='2017')

Podemos notar neste gráfico que existe uma concentração maior no período de férias escolares, entre dezembro e fevereiro. Também é interessante notar algumas 'linhas' em datas específicas, isso pode ser explciado com algum feríado ou possívelmente algum problema que tenha ocorrido no aeroporto. Ex:

 - Final de ferereiro - onde acontece o carnaval (dia 28), 
 - Final de abril que pode ser explicado pelo dia do trabalhador em primeiro de maio (segunda feira)

### Histograma de atrasos de partida

In [ ]:
hist(as.numeric(atrasos$atraso_partida)
     ,breaks=seq(15,240,15)
     ,ylim=c(0,0.04)
     ,xlim=c(0,250)
     ,xlab="atraso (min)"
     ,main="Voos atrasados"
     ,freq=FALSE
     )

#### Histograma de horas mais comuns de voos com atraso

In [ ]:
deh = as.POSIXlt(atrasos$partida_prevista)

h=hist(deh$hour+deh$min/60
     ,breaks=seq(0, 24, 0.5)
     ,xlim=c(0,25)
     ,xlab="Hour of the day"
     ,main="Most frequent departure expected hours"
     )
rm(deh)

#### Horas do dia com maior atraso (** tem que ajustar o gráfico, tem a janela de 1 da manhã que é pulada**)

Para essa análise foi usado os dados inclusive que tiveram atraso, apenas foi retirado os voos adiantados (atraso negativo), pois essa análise tem como objetivo encontrar quais horários são mais propensos a atraso, para entender qual horário deve ser evitado pelo passageiro na hora de comprar uma passagem.

A fim de evitar atrasos tentamos entender melhor qual hora do dia os voos costumam atrasar mais, para isso pegamos os horários de partida previsto e verificamos em média quanto tempo voos nessa hora costumam atrasar

In [ ]:
library(dplyr) #%>%
d_h = vrawu17[,c('partida_prevista','atraso_partida')]
d_h = d_h[rowSums(is.na(d_h))==0,]
d_h = d_h[d_h$atraso_partida>=0,]

d_h$por_hora_tempo_partida=cut(as.integer(format(d_h$partida_prevista, "%H")), seq(0,24,1), include.lowest=TRUE, labels=seq(0,23,1))  #seq(0,23,1) )
d_h = d_h %>% group_by(por_hora_tempo_partida) %>% summarize(avg = mean(atraso_partida))

In [ ]:
barplot(as.numeric(d_h$avg),names.arg=seq(0,22,1), ylim=c(0,60), ylab='atraso (min)')
rm(d_h)

### Densidade de distribuição PROBLEMA


In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
library(RColorBrewer)
library(gridExtra)

col.set <- brewer.pal(11, 'Spectral')
mycolors <- col.set[seq(0,36,1)]#c(1,3,5,7,9,11)]

value=vrawu17[vrawu17$atraso_partida >=15 & vrawu17$atraso_partida <=240,]
series <- data.frame(variable=value$origem_condicoes, value=value$atraso_partida)
grfB <- plot.density(series, colors=mycolors)

options(repr.plot.width=10, repr.plot.height=4)
grid.arrange( grfB, ncol=2)
#summary(vrawu17)

In [ ]:
vrawu17$origem_condicoes[1:100]
vrawu17$origem_condicoes[96] == "\xc2"

In [ ]:
library(stringr)
usableText=str_replace_all(vrawu17$origem_condicoes,"[^[:graph:]]", "NA")

In [ ]:
delays = vrawu17[vrawu17$atraso_partida >=0  & vrawu17$atraso_partida <= 240,]
options(repr.plot.width=4, repr.plot.height=3)
qqplot(delays$atraso_partida, delays$origem_visibilidade)
fm <- lm(delays$atraso_partida ~ delays$origem_visibilidade)
abline(coef(fm), lty=4)

options(repr.plot.width=4, repr.plot.height=3)
qqplot(delays$atraso_partida, delays$origem_pressao)
fm <- lm(delays$atraso_partida ~ delays$origem_pressao)
abline(coef(fm), lty=4)

options(repr.plot.width=4, repr.plot.height=3)
qqplot(delays$atraso_partida, delays$origem_dew_point)
fm <- lm(delays$atraso_partida ~ delays$origem_dew_point)
abline(coef(fm), lty=4)

options(repr.plot.width=4, repr.plot.height=3)
qqplot(delays$atraso_partida, delays$origem_temperatura)
fm <- lm(delays$atraso_partida ~ delays$origem_temperatura)
abline(coef(fm), lty=4)

### Mapa do Brasil
Fonte: https://downloads.ibge.gov.br/downloads_geociencias.htm

Primeiro temos que obter os dados de localização dos aeroportos

In [ ]:
message=FALSE
library(data.table)
library(dplyr) #select %>%
airport = fread("dataset/airports_simplified.csv")

br_airport = select(airport[country == 'Brazil'],ICAO) 

vw = merge( atrasos, br_airport, by.x = "icao_destino", by.y = "ICAO")
vw = merge( atrasos, br_airport, by.x = "icao_origem", by.y = "ICAO")


vw = vw[c("icao_origem", "icao_destino")]
vw = vw %>% group_by(icao_destino,icao_origem) %>% summarize(Count = n())

library(tidyverse)
vw$icao_destino = as.character(vw$icao_destino)
vw$icao_origem = as.character(vw$icao_origem)
vw = vw %>%
    rowwise() %>%
    mutate(grp = paste0(sort(c(icao_destino, icao_origem)), collapse = "_")) %>%
    ungroup() %>%
    group_by(grp) %>%
    summarise(Count = sum(Count)) %>%
    separate(grp, into = c("icao_destino", "icao_origem"))

In [ ]:
#Join Departure
vra_wu<-vw
vra_wu= merge(vra_wu,airport, by.x="icao_destino", by.y="ICAO")
colnames(vra_wu)[colnames(vra_wu) == "name"] = "destiny_airport_name"
colnames(vra_wu)[colnames(vra_wu) == "city"] = "destiny_city"
colnames(vra_wu)[colnames(vra_wu) == "country"] = "destiny_country"
colnames(vra_wu)[colnames(vra_wu) == "latitude"] = "destiny_latitude"
colnames(vra_wu)[colnames(vra_wu) == "longitude"] = "destiny_longitude"

#Join Origin
vra_wu= merge(vra_wu,airport, by.x="icao_origem", by.y="ICAO")
colnames(vra_wu)[colnames(vra_wu) == "name"] = "origin_airport_name"
colnames(vra_wu)[colnames(vra_wu) == "city"] = "origin_city"
colnames(vra_wu)[colnames(vra_wu) == "country"] = "origin_country"
colnames(vra_wu)[colnames(vra_wu) == "latitude"] = "origin_latitude"
colnames(vra_wu)[colnames(vra_wu) == "longitude"] = "origin_longitude"

In [ ]:
library(ggplot2)
library(maptools)


mapa <- borders(database ="world", regions = "Brazil", fill = "grey70", colour = "black")

brazil <- ggplot() + mapa + theme_bw() + xlab("Longitude (decimals)") + ylab("Latitude (decimals)") + 
  theme(panel.border = element_blank(), panel.grid.major = element_line(colour = "grey80"), panel.grid.minor = element_blank())

estados <- readShapePoly("dataset/br_unidades_da_federacao/BRUFE250GC_SIR.shp")
estados1 <- fortify(estados)

br_est <- brazil + geom_path(data = estados1, aes(x = long, y = lat, group = group), colour = "black")
br_est

In [ ]:
#library(tidyverse)
#library(maps)
library(geosphere)

vra_wu = vra_wu[vra_wu$icao_origem == 'SBGR',]

par(mar=c(0,0,0,0))
#map('world','Brazil',col="#f2f2f2", fill=TRUE, bg="white", lwd=0.05,mar=rep(0,4),border=0, resolution=0)
pal <- colorRampPalette(c("#666666", "black"))
colors <- pal(30)

max_cnt = max(vra_wu$Count)
min_cnt = min(vra_wu$Count)
lwd_max = 5
lwd_min = 0.8


for (j in 1:dim(vra_wu)[1]) {
    #j=15*i
    origin = c(vra_wu$origin_longitude[j],vra_wu$origin_latitude[j])
    destiny = c(vra_wu$destiny_longitude[j],vra_wu$destiny_latitude[j])
     
    inter = as.data.frame(gcIntermediate(origin, destiny, n=100, addStartEnd=TRUE))
    pu = (vra_wu$Count[j] -min_cnt)/(max_cnt-min_cnt)
    col_index =1+ round(pu*(length(colors)-1))
    lin = pu*(lwd_max-lwd_min)+lwd_min
    
    names(inter) <- c("lon", "lat")
    br_est <- br_est + geom_line(data=inter, aes(x=lon, y=lat), color='#FFFFFF')
    
    #br_est <- br_est + inter
    #lines(inter, col=colors[col_index],lwd=lin)
    #lines(inter, col="#666666",lwd=0.5)
    #min=0.8
    #if (i==100){break}
}